In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

앙상블 학습은 크게 독립적 앙상블과 의존적 앙상블로 나눌 수 있다.

독립적 앙상블(Independent Ensenble)  
독립적 앙상블 방법에서 사용하는 개별 분류기들은 서로 독립적인 알고리즘이므로 각 분류기는 서로 다른 머신러닝 알고리즘을 사용할 수 있다. 각 분류기가 독립적이므로 효과적으로 병렬화 할 수 있다는 장점이 있다.

의존적 앙상블(Dependent Ensenble)  
의존적 앙상블 방법은 독립적 앙상블 방법과는 달리 개별 학습기들이 서로 독립이 아닌 경우를 의미하며 의존적 앙상블 방법 중 가장 유명한 방법은 부스팅이다.

배깅(Bagging, Bootstrap aggregating)과 랜덤 포레스트(Random Forest)  
배깅은 개별 분류기들의 결과를 종합하여 최종 분류기의 성능을 향상시키는 방법이다.  
개별 분류기들이 동일한 트레이닝 데이터로 학습하는 보팅과는 달리 배깅은 오리지널 트레이닝 데이터셋에서 샘플(부트스트랩)을 뽑아 학습한다. 부트스트랩이란 중복을 허용한 랜덤 샘플 방식을 의미하고 개별 분류 모형의 결과값을 모아 다수결 투표를 통해 최종 예측하게 된다.  
랜덤 포레스트는 여러 개의 개별 분류기인 의사 결정 트리를 토대로 예측한 결과를 종합해 전체 예측 정확도를 높이는 방법이다.

랜덤 포레스트 알고리즘을 활용해 와인 데이터를 분류하는 모델을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_wine = datasets.load_wine() # 와인 데이터를 불러온다.

In [4]:
# 피쳐, 타겟 데이터 저장
X = raw_wine.data # 피쳐 데이터를 저장한다.
y = raw_wine.target # 타겟 데이터를 저장한다.
X.shape

(178, 13)

In [5]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(133, 13) (133,) (45, 13) (45,)


In [6]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [66]:
# 모델 생성 후 데이터 학습
from sklearn.ensemble import RandomForestClassifier # 랜덤 포레스트 알고리즘을 사용하기 위해 import 한다.
# 너무 깊게 학습하면 과적합이 발생될 수 있으므로 max_depth 옵션을 지정해서 과적합을 방지한다.
clf_rf = RandomForestClassifier(max_depth=2, random_state=0) # 랜덤 포레스트 모델을 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 랜덤 포레스트 알고리즘을 학습시킨다.
clf_rf.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=2, random_state=0)

In [67]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_rf = clf_rf.predict(X_te_std)
print(pred_rf)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [68]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_rf)
print(accuracy)

0.9555555555555556


In [69]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_rf, average=None)
print(precision)

[0.94117647 1.         0.88888889]


In [70]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_rf, average=None)
print(recall)

[1.        0.9047619 1.       ]


In [71]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_rf, average=None)
print(f1)

[0.96969697 0.95       0.94117647]


In [72]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_rf)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


In [75]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_rf)를 넘겨 분류 리포트를 만든다.
target_names = raw_wine.target_names
class_report = classification_report(y_te, pred_rf, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

     class_0       0.94      1.00      0.97        16
     class_1       1.00      0.90      0.95        21
     class_2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45

